In [1]:
# Import statements
import pandas as pd

In [2]:
# Reading in data
mushroom_baseDF= pd.read_csv('mushroom_overload.csv', low_memory=False)

In [3]:
# Getting a look at the different columns available
mushroom_baseDF.columns

Index(['class', 'cap-diameter', 'cap-shape', 'cap-surface', 'cap-color',
       'does-bruise-or-bleed', 'gill-attachment', 'gill-spacing', 'gill-color',
       'stem-height', 'stem-width', 'stem-root', 'stem-surface', 'stem-color',
       'veil-type', 'veil-color', 'has-ring', 'ring-type', 'spore-print-color',
       'habitat', 'season'],
      dtype='object')

In [4]:
for i in mushroom_baseDF.columns:
    print(f"Number of null-values in {i}: {mushroom_baseDF[i].isnull().sum().sum()}")

Number of null-values in class: 0
Number of null-values in cap-diameter: 0
Number of null-values in cap-shape: 0
Number of null-values in cap-surface: 1579337
Number of null-values in cap-color: 0
Number of null-values in does-bruise-or-bleed: 0
Number of null-values in gill-attachment: 1066721
Number of null-values in gill-spacing: 2800709
Number of null-values in gill-color: 0
Number of null-values in stem-height: 0
Number of null-values in stem-width: 0
Number of null-values in stem-root: 5763499
Number of null-values in stem-surface: 4263506
Number of null-values in stem-color: 0
Number of null-values in veil-type: 6367343
Number of null-values in veil-color: 5893024
Number of null-values in has-ring: 0
Number of null-values in ring-type: 276729
Number of null-values in spore-print-color: 6049365
Number of null-values in habitat: 0
Number of null-values in season: 0


In [5]:
# Slimming down the dataframe into a new one to only have the relavent data
mushroom_DF = mushroom_baseDF[['cap-shape', 'cap-surface', 'stem-root', 'ring-type', 'season', 'habitat']]
mushroom_DF.head()

,cap-shape,cap-surface,stem-root,ring-type,season,habitat
0,f,g,s,g,a,d
1,x,h,s,g,u,d
2,f,h,s,p,a,d
3,f,g,s,g,u,d
4,x,h,s,p,w,d


In [6]:
data_dict = {'cap-shape': {'b': 'bell', 'c': 'conical', 'x': 'convex', 'f': 'flat', 's': 'sunken', 'p': 'spherical', 'o': 'others'},
             'cap-surface': {'i': 'fibrous', 'g': 'grooves', 'y': 'scaly', 's': 'smooth', 'd': 'dry', 'h': 'shiny', 'l': 'leathery', 'k': 'sikly', 
                             't': 'sticky', 'w': 'wrinkled', 'e': 'fleshy'},
             'stem-root': {'b': 'bulbous', 's': 'swollen', 'c': 'club', 'u': 'cup', 'e': 'equal', 'z': 'rizhomorphs', 'r': 'rooted'},
             'ring-type': {'c': 'cobwebby', 'e': 'evanexcent', 'r': 'flaring', 'g': 'grooved', 'l': 'large', 'p': 'pendant', 's': 'sheathing', 
                           'z': 'zone', 'y': 'scaly', 'm': 'movable', 'f': 'none', '?': 'unknown'},
             'season': {'s': 'spring', 'u': 'summer', 'a': 'autumn', 'w': 'winter'},
             'habitat': {'g': 'grasses', 'l': 'leaves', 'm': 'meadows', 'p': 'paths', 'h': 'heaths', 'u': 'urban', 'w': 'waste', 'd': 'woods'}}

In [9]:
mushroom_baseDF.shape[0] - 5763499

959617

Looking into how many rows do I still have if I drop rows with NaN values for all the selected columns